In [ ]:
!pip install -q transformers datasets deepspeed torch accelerate evaluate bitsandbytes pyyaml wandb

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
WANDB_API_KEY = user_secrets.get_secret("WANDB_API_KEY")

In [ ]:
import wandb
wandb.login(key=WANDB_API_KEY)

In [ ]:
import os
import yaml
import json

# --- Tạo thư mục cấu hình cho Accelerate ---
accelerate_config_dir = os.path.expanduser("~/.cache/huggingface/accelerate")
os.makedirs(accelerate_config_dir, exist_ok=True)

# --- Định nghĩa cấu hình Accelerate ---
accelerate_config = {
    'compute_environment': 'LOCAL_MACHINE',
    'distributed_type': 'DEEPSPEED',
    'deepspeed_config': {
        'deepspeed_config_file': '/kaggle/working/ds_zero3_config.json',
        'zero3_init_flag': True, 
    },
    'machine_rank': 0,
    'main_training_function': 'main',
    'num_machines': 1,
    'num_processes': 2,
    'use_cpu': False,
}

# --- Ghi file cấu hình Accelerate ---
config_path = os.path.join(accelerate_config_dir, "default_config.yaml")
with open(config_path, 'w') as f:
    yaml.dump(accelerate_config, f, default_flow_style=False)
print(f"File cấu hình Accelerate đã được tạo tại: {config_path}")

# --- Định nghĩa cấu hình DeepSpeed ---
deepspeed_config = {
  "fp16": {
    "enabled": "auto"
  },
  "bf16": {
    "enabled": "auto"
  },
  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": 3e-6,
      "betas": [0.9, 0.999],
      "eps": 1e-8,
      "weight_decay": 0.01
    }
  },
  "scheduler": {
    "type": "WarmupDecayLR",
    "params": {
      "total_num_steps": 183,
      "warmup_min_lr": 0,
      "warmup_max_lr": 3e-6,
      "warmup_num_steps": 20
    }
  },
  "zero_optimization": {
    "stage": 3,
    "offload_optimizer": { "device": "cpu", "pin_memory": True },
    "offload_param": { "device": "cpu", "pin_memory": True },
    "overlap_comm": True,
    "contiguous_gradients": True,
    "sub_group_size": 1e9,
    "reduce_bucket_size": "auto",
    "stage3_prefetch_bucket_size": "auto",
    "stage3_param_persistence_threshold": "auto",
    "stage3_max_live_parameters": 1e9,
    "stage3_max_reuse_distance": 1e9,
    "stage3_gather_16bit_weights_on_model_save": True
  },
  "gradient_accumulation_steps": "auto",
  "gradient_clipping": "auto",
  "steps_per_print": 2000,
  "train_batch_size": "auto",
  "train_micro_batch_size_per_gpu": "auto",
  "wall_clock_breakdown": False
}

# --- Ghi file cấu hình DeepSpeed ---
ds_config_path = "/kaggle/working/ds_zero3_config.json"
with open(ds_config_path, 'w') as f:
    json.dump(deepspeed_config, f, indent=2)
print(f"File cấu hình DeepSpeed đã được sửa và tạo tại: {ds_config_path}")

In [ ]:
%%writefile train_deepspeed_wandb.py

import torch
import time
import math
import argparse
import wandb
from accelerate.utils import DummyOptim, DummyScheduler
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator
from datasets import load_dataset
from torch.utils.data import DataLoader
from accelerate import Accelerator
from tqdm.auto import tqdm

def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def evaluate_model(model, dataloader, accelerator, args):
    model.eval()
    losses = []
    eval_start_time = time.time()
    for batch in dataloader:
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        losses.append(accelerator.gather_for_metrics(loss.repeat(args.batch_size)))

    losses = torch.cat(losses)
    if accelerator.num_processes > 1:
        losses = losses[:len(dataloader.dataset)]
    try:
        eval_loss = torch.mean(losses)
        perplexity = math.exp(eval_loss)
    except OverflowError:
        eval_loss = torch.tensor(float("inf"))
        perplexity = float("inf")
    
    eval_time = time.time() - eval_start_time
    model.train()
    return eval_loss.item(), perplexity, eval_time

def main():
    # Command arguments
    parser = argparse.ArgumentParser(description="Finetune BLOOM with DeepSpeed ZeRO-3 and W&B")
    
    parser.add_argument("--model_name", type=str, default="bigscience/bloom-560m")
    parser.add_argument("--dataset_name", type=str, default="Salesforce/wikitext")
    parser.add_argument("--dataset_config", type=str, default="wikitext-2-raw-v1")
    
    parser.add_argument("--seed", type=int, default=42)
    parser.add_argument("--num_epochs", type=int, default=2)
    parser.add_argument("--batch_size", type=int, default=4)
    parser.add_argument("--block_size", type=int, default=128)
    
    parser.add_argument("--logging_steps", type=int, default=5)
    parser.add_argument("--eval_steps", type=int, default=20)
    parser.add_argument("--wandb_project", type=str, default="deepspeed_bloom_finetune")
    parser.add_argument("--wandb_run_name", type=str, default=None)
    
    args = parser.parse_args()
    
    if args.wandb_run_name:
        args.wandb_run_name += f"-{int(time.time())}"

    # Randome seed
    set_seed(args.seed)

    # Init accelerator
    accelerator = Accelerator(log_with="wandb")

    # Init wandb run
    if accelerator.is_main_process:
        wandb.init(project=args.wandb_project, name=args.wandb_run_name, config=vars(args))

    # Display command arguments
    accelerator.print("Arguments:", args)

    # General setup completed
    accelerator.print(f"Đang sử dụng {accelerator.num_processes} GPUs với DeepSpeed.")

    # Model & tokenizer
    accelerator.print("Đang tải tokenizer và model...")
    tokenizer = AutoTokenizer.from_pretrained(args.model_name)
    model = AutoModelForCausalLM.from_pretrained(args.model_name)
    
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id

    # Dataset
    accelerator.print("Đang tải và xử lý dữ liệu...")
    raw_datasets = load_dataset(args.dataset_name, args.dataset_config)
    raw_datasets['train'] = raw_datasets['train'].select(range(1000))
    raw_datasets['validation'] = raw_datasets['validation'].select(range(100))
    del raw_datasets['test']
    
    column_names = raw_datasets["train"].column_names
    text_column_name = "text" if "text" in column_names else column_names[0]

    # Tokenize text
    def tokenize_function(examples):
        return tokenizer(examples[text_column_name])

    tokenized_datasets = raw_datasets.map(
        tokenize_function, batched=True, remove_columns=column_names, desc="Running tokenizer on dataset"
    )
    accelerator.print(tokenized_datasets)

    # Chunking
    def group_texts(examples):
        concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        total_length = (total_length // args.block_size) * args.block_size
        result = {
            k: [t[i : i + args.block_size] for i in range(0, total_length, args.block_size)]
            for k, t in concatenated_examples.items()
        }
        result["labels"] = result["input_ids"].copy()
        return result

    lm_datasets = tokenized_datasets.map(
        group_texts, batched=True, desc=f"Grouping texts in chunks of {args.block_size}"
    )
    accelerator.print(lm_datasets)
    
    train_dataset = lm_datasets["train"]
    eval_dataset = lm_datasets["validation"]

    # Data collator
    data_collator = default_data_collator
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=args.batch_size, collate_fn=data_collator)
    eval_dataloader = DataLoader(eval_dataset, batch_size=args.batch_size, collate_fn=data_collator)

    # Optimizer & scheduler
    optimizer = DummyOptim(model.parameters())
    scheduler = DummyScheduler(optimizer)

    model, optimizer, train_dataloader, eval_dataloader, scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader, scheduler
    )

    # Training
    accelerator.print("\n*** Bắt đầu huấn luyện ***")
    global_step = 0
    start_training_time = time.time()
    
    for epoch in range(args.num_epochs):
        model.train()
        progress_bar = tqdm(
            train_dataloader,
            desc=f"Epoch {epoch+1}/{args.num_epochs}",
            disable=not accelerator.is_local_main_process
        )
        start_epoch_time = time.time()
        for batch in progress_bar:
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
            
            if global_step % args.logging_steps == 0:
                learning_rate = scheduler.get_last_lr()[0]
                log_metrics = {
                    "train_loss": loss.item(),
                    "learning_rate": learning_rate,
                    "epoch": global_step / len(train_dataloader),
                }
                if accelerator.is_main_process: wandb.log(log_metrics, step=global_step)

            if global_step % args.eval_steps == 0:
                eval_loss, perplexity, eval_time = evaluate_model(model, eval_dataloader, accelerator, args)
                log_metrics = {
                    "eval_loss": eval_loss,
                    "perplexity": perplexity,
                    "eval_time (s)": eval_time,
                }
                if accelerator.is_main_process: wandb.log(log_metrics, step=global_step)
                accelerator.print(f"Step {global_step}: eval_loss = {eval_loss:.2f}")

            if accelerator.is_local_main_process:
                progress_bar.set_postfix({"loss": loss.item(), "step": global_step})
        
        log_metrics = {
            "epoch_time (s)": time.time() - start_epoch_time,
            "epoch": epoch + 1,
        }
        if accelerator.is_main_process: wandb.log(log_metrics, step=global_step)

    # Total training time
    accelerator.wait_for_everyone()
    total_training_time = time.time() - start_training_time
    accelerator.print(f"*** Huấn luyện hoàn tất trong: {total_training_time:.2f} giây ***\n")
    
    # Final evaluation
    accelerator.print("*** Bắt đầu đánh giá cuối cùng ***")
    final_eval_loss, final_perplexity, _ = evaluate_model(model, eval_dataloader, accelerator, args)

    if accelerator.is_main_process:
        print(f"*** Kết quả đánh giá cuối cùng trên tập validation ***")
        print(f"Epoch: {args.num_epochs}")
        print(f"Loss: {final_eval_loss:.4f}")
        print(f"Perplexity: {final_perplexity:.4f}")
    
    accelerator.end_training()
    wandb.finish()

if __name__ == "__main__":
    main()

In [ ]:
!accelerate launch train_deepspeed_wandb.py \
    --model_name "bigscience/bloom-560m" \
    --batch_size 4 \
    --num_epochs 3 \
    --logging_steps 2 \
    --eval_steps 10 \
    --wandb_project "PARADIS-bloom_560m" \
    --wandb_run_name "ZeRO_3"